In [3]:
import torch

In [4]:
from graph_generation_trade import dict_snapshots_trade, temporal_data_trade

from models_and_utils import (
    data_split, train_snapshots, test_snapshots, apply_negative_sampling
)

In [3]:
from models_and_utils import ModelMPNN

data = dict_snapshots_trade
data_train, data_test = data_split(graph_snapshots=data, test_ratio=0.8)

in_channels = data[0].x.shape[1]
hidden_size = 128
num_nodes = data[0].x.shape[0]
window = 1
dropout_p = 0.2

model = ModelMPNN(in_channels, hidden_size, num_nodes, window, dropout_p)

lr = 0.00005
neg_sampling_rate=0.1
epochs = 40 
optimiser = torch.optim.Adam(model.parameters(), lr=lr)

results = train_snapshots(model, data_train, optimiser, epochs, neg_sampling_rate)
results['loss'].plot()
plt.show()

(TemporalData(src=[4468], dst=[4468], t=[4468], msg=[4468]),
 {Timestamp('2007-02-01 00:00:00'): tensor([ 7.7705, 10.4309,  9.5677,  7.0494,  6.7228,  9.1620, 14.3490,  5.9417,
           5.2618, 13.9241,  7.9439,  6.0080, 10.5008,  9.9177]),
  Timestamp('2007-03-01 00:00:00'): tensor([ 6.3703, 11.4182,  9.7674,  6.3559,  7.5576,  9.2053, 14.3388,  7.0861,
           5.3565, 14.0265,  8.2337, 10.3065, 10.5581,  9.2569]),
  Timestamp('2007-04-01 00:00:00'): tensor([ 7.5199, 10.7455, 10.1090,  7.9813,  7.5959,  9.2408, 14.1979,  7.6520,
           4.6322, 14.0517,  8.7288,  9.8865,  9.9969, 10.6283]),
  Timestamp('2007-05-01 00:00:00'): tensor([ 6.8922, 11.5264,  9.6466,  7.4341,  7.6010,  9.4389, 14.2460,  6.2641,
           5.8373, 14.1184,  8.4750,  6.7442, 10.3591, 10.2048]),
  Timestamp('2007-06-01 00:00:00'): tensor([ 7.5880, 11.0147,  9.8529,  7.2671,  8.2087,  9.1291, 14.4522,  6.4047,
           5.9813, 14.0811,  8.0324,  8.8421, 10.1687, 10.6290]),
  Timestamp('2007-07-01 00:00